In [163]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import snowflake.connector
from model_orm import URLOrm, Base
from models import URLClass
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

True

### ORM CLASS


In [164]:
from sqlalchemy import Column, Integer, String, Sequence
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class URLOrm(Base):
    __tablename__ = 'url_data1'

    id = Column(Integer, Sequence('url_table1_id_seq'), primary_key=True, autoincrement=True)
    name_of_the_topic = Column(String)
    year = Column(String)
    level = Column(String)
    topics = Column(String)
    learning_outcomes = Column(String, default="Not Available")
    introduction = Column(String, default="Not Available")
    summary = Column(String, default="Not Available")
    link_to_the_pdf_file = Column(String, default="/")
    link_to_the_summary_page = Column(String, default="https://")

### LOAD ENVIRONMENT VARIABLES

In [165]:
user = os.getenv('SNOWFLAKE_USER')
password = os.getenv('SNOWFLAKE_PASSWORD')
account = os.getenv('SNOWFLAKE_ACCOUNT')
warehouse = os.getenv('SNOWFLAKE_WAREHOUSE')
database = os.getenv('SNOWFLAKE_DATABASE')
schema = os.getenv('SNOWFLAKE_SCHEMA')

### Read and Pre-Process Data

In [166]:
def read_csv(csv_path):
    return pd.read_csv(csv_path)

def process_data(df):
    # Replace NaN values with None for compatibility with Snowflake
    # df = df.where(pd.notnull(df), None)
    df=df.fillna('')
    # df = df.fillna('')
    df['Year'] = df['Year'].replace('', '2024')
    df['Year'] = df['Year'].astype(str).str.extract('(\d+)').astype(float).astype(int)
    # Stripping "Curriculum" from the 'Curriculum' column values
    # df['Year'] = df['Year'].str.replace('Curriculum', '').str.strip()
    
    # Stripping "CFA Program" from the 'Level' column values
    df['Level'] = df['Level'].str.replace('CFA Program', '').str.strip()
    return df

# Example usage:
csv_path = '../resources/cfa_url/cfa_list_input.csv'
df = read_csv(csv_path)
df_processed = process_data(df)


In [167]:
df_processed.isna().sum()

Name of the topic            0
Year                         0
Level                        0
Topics                       0
Learning Outcomes Section    0
Introduction                 0
Summary Bullets              0
Link to the PDF File         0
Link to the Summary Page     0
dtype: int64

In [168]:
# df_processed.isna().sum()


### Save Processed file 

In [169]:
import os

processed_csv_path = '../resources/clean_csv/processed_content.csv'

# Ensure the directory exists; if not, create it
os.makedirs(os.path.dirname(processed_csv_path), exist_ok=True)

# Save the processed DataFrame to CSV

print(f'Processed DataFrame saved to {processed_csv_path}')

Processed DataFrame saved to ../resources/clean_csv/processed_content.csv


In [170]:
tp='prc.csv'
df_processed.to_csv(tp, index=False)


In [171]:
df_processed.isna().sum()


Name of the topic            0
Year                         0
Level                        0
Topics                       0
Learning Outcomes Section    0
Introduction                 0
Summary Bullets              0
Link to the PDF File         0
Link to the Summary Page     0
dtype: int64

In [172]:
# l=pd.read_csv(tp)
# l.isna().sum()

### Create ORM INSTANCE

In [173]:
def create_orm_instances(df):
    orm_instances = []
    for _, row in df.iterrows():
        orm_instance = URLOrm(
            name_of_the_topic=row['Name of the topic'],
            year=str(row['Year']),  
            level=row['Level'],
            topics=row['Topics'],
            learning_outcomes=row['Learning Outcomes Section'],
            introduction=row['Introduction'],
            summary=row['Summary Bullets'],
            link_to_the_pdf_file=row['Link to the PDF File'],
            link_to_the_summary_page=row['Link to the Summary Page']
        )
        orm_instances.append(orm_instance)
    return orm_instances


### Use Pydantic to Validate

In [174]:
from pydantic import BaseModel, constr, Field, validator
from typing import Optional, Any
from typing_extensions import Annotated

class URLClass(BaseModel):
    # id: int
    name_of_the_topic: Annotated[str, Field(alias='name_of_the_topic')]
    year: int = Field(..., ge=1900, le=2024)
    level: Annotated[str, Field(alias='level')]
    topics: constr(strict=True, min_length=1)
    learning_outcomes: Annotated[str, Field(alias='learning_outcomes')]
    introduction: Annotated[Optional[str], Field(alias='Introduction', default="Not Available")]
    summary: Annotated[Optional[str], Field(alias='Summary', default="Not Available")]
    link_to_the_pdf_file: Annotated[Optional[str], Field(alias='Link to the PDF file',default="/")]
    link_to_the_summary_page: Annotated[Optional[str], Field(alias='Link to the Summary Page', default="/")]


    @validator('year')
    def validate_curriculum_year(cls, value):
        if value < 1900 or value > 2024:
            raise ValueError('Curriculum year must be between 1900 and 2024')
        return value

    @validator('level')
    def validate_level(cls, value):
        if not value.startswith('Level'):
            raise ValueError('Level must start with the keyword "Level"')
        return value

    @validator('learning_outcomes', 'introduction', 'summary', pre=True)
    def default_not_available_if_empty(cls, value):
        return value if value else "Not Available"

    @validator('link_to_the_pdf_file')
    def validate_pdf_link(cls, value):
        if not value.endswith('.pdf'):
            raise ValueError('PDF link must end with .pdf')
        return value

    @validator('link_to_the_summary_page')
    def validate_pdf_link(cls, value):
        if not value.startswith('https://'):
            raise ValueError('Link to summary page must start with https://')
        return value
    class Config:
        anystr_strip_whitespace = True



/var/folders/6q/q891flcj0r375hpjwrt2wtbm0000gn/T/ipykernel_64126/3012125672.py:18: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  @validator('year')
/var/folders/6q/q891flcj0r375hpjwrt2wtbm0000gn/T/ipykernel_64126/3012125672.py:24: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  @validator('level')
/var/folders/6q/q891flcj0r375hpjwrt2wtbm0000gn/T/ipykernel_64126/3012125672.py:30: PydanticDeprecatedSince20: Pydantic V1 style `@validat

### To convert ORM instance to Pydantic

In [175]:

from pydantic import BaseModel, Field, validator, ValidationError

def orm_instance_to_pydantic(orm_instance):
    try:
        return URLClass(**orm_instance.__dict__)
    except ValidationError as e:
        # Handle or log the validation error
        print(f"Validation error for {orm_instance.name_of_the_topic}: {e}")

def convert_to_pydantic_instances(orm_instances):
    # Removed 'self' since this function is used as a standalone
    return [orm_instance_to_pydantic(orm_instance) for orm_instance in orm_instances]



In [176]:
def create_orm_instances_and_validate_using_pydantic(path):
    try:
        df = pd.read_csv(path)
        df=df.fillna('')
        orm_instances = create_orm_instances(df)
        # for i in orm_instances:
        #     print(i.name_of_the_topic)
        pydantic_instances = convert_to_pydantic_instances(orm_instances)
        print(f"{len(pydantic_instances)} are validated")
        return orm_instances
    except Exception as e:
        print(str(e))
        print("Error in validation")

In [177]:
orm_instances = create_orm_instances_and_validate_using_pydantic(processed_csv_path)

224 are validated


### Upload to Snowflake

In [178]:
def create_database_if_not_exists(engine, database):
    connection = engine.connect()
    connection.execute(f"CREATE DATABASE IF NOT EXISTS {database}")
    connection.close()

In [179]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.schema import CreateTable

Base = declarative_base()

def upload_to_snowflake(engine, orm_instances, database):
    create_database_if_not_exists(engine, database)
    Base.metadata.bind = engine

    if not engine.dialect.has_table(engine, URLOrm.__tablename__):  
        URLOrm.__table__.create(bind=engine) 
    else:
        print(f"Table '{URLOrm.__tablename__}' already exists.")  
    
    SessionClass = sessionmaker(bind=engine)
    session = SessionClass()

    total_instances = len(orm_instances)
    progress_threshold = 10  

    for idx, orm_instance in enumerate(orm_instances, start=1):
        session.add(orm_instance)
        if idx % progress_threshold == 0 or idx == total_instances:
            session.commit()
            print(f"Inserted {idx}/{total_instances} records.")

    session.close()
    print("All records have been inserted.")


In [180]:
engine = create_engine(
    f'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'
)
upload_to_snowflake(engine=engine, orm_instances=orm_instances, database=database)

Table 'url_data1' already exists.
Inserted 10/224 records.
Inserted 20/224 records.
Inserted 30/224 records.
Inserted 40/224 records.
Inserted 50/224 records.
Inserted 60/224 records.
Inserted 70/224 records.
Inserted 80/224 records.
Inserted 90/224 records.
Inserted 100/224 records.
Inserted 110/224 records.
Inserted 120/224 records.
Inserted 130/224 records.
Inserted 140/224 records.
Inserted 150/224 records.
Inserted 160/224 records.
Inserted 170/224 records.
Inserted 180/224 records.
Inserted 190/224 records.
Inserted 200/224 records.
Inserted 210/224 records.
Inserted 220/224 records.
Inserted 224/224 records.
All records have been inserted.
